<a href="https://colab.research.google.com/github/ArtemFilosofGB/ColabPractic/blob/main/BigData_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

Урок 2. SQL & Big Data
Условие:
Загрузите датасет по ценам на жилье Airbnb, доступный на kaggle.com: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

* Подсчитайте среднее значение и дисперсию по признаку ”price” в hive
Используя Python,

* реализуйте скрипт mapper.py и reducer.py для расчета

* Проверьте правильность подсчета статистики методом mapreduce в сравнении со hive.

In [3]:
dataset_path = Path('/content/', 'AB_NYC_2019.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data')


In [9]:
db= pd.read_csv('AB_NYC_2019.csv', delimiter=',')
db.head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [14]:
db[['name','price']]

,name,price
0,Clean & quiet apt home by the park,149
1,Skylit Midtown Castle,225
2,THE VILLAGE OF HARLEM....NEW YORK !,150
3,Cozy Entire Floor of Brownstone,89
4,Entire Apt: Spacious Studio/Loft by central park,80
...,...,...
48890,Charming one bedroom - newly renovated rowhouse,70
48891,Affordable room in Bushwick/East Williamsburg,40
48892,Sunny Studio at Historical Neighborhood,115
48893,43rd St. Time Square-cozy single bed,55


In [18]:
db[['price']].var() #дисперсия по значению

price    57674.025247
dtype: float64

In [20]:
db[['price']].std() #стандартное отклонение

price    240.15417
dtype: float64

In [ ]:
# Через multiprocessing
import multiprocessing as mp
from functools import reduce


result_list = []
def log_result(result):
    result_list.append(result)

def count_songs(chunk):
    return chunk[(chunk.year == 2022) & (chunk.views > 5000)].shape[0]

def apply_async_with_callback(chunksize):
    pool = mp.Pool(2)
    for i in pd.read_csv(dataset_path, chunksize=chunksize, iterator=True):
        pool.apply_async(count_songs, args = (i, ), callback = log_result)
    pool.close()
    pool.join()
    return result_list

if __name__ == '__main__':
    chunksize = 100_000
    start_time = time.time()
    print(reduce(lambda x, y: x + y, apply_async_with_callback(chunksize)))
    print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.')

2650
Время выполнения: 246.4 секунд.


In [ ]:
from functools import reduce


def count_songs(chunk):
    return chunk[(chunk.year == 2022) & (chunk.views > 5000)].shape[0]


chunksize = 100_000
df_chunks = pd.read_csv(
    dataset_path,
    chunksize=chunksize)

start_time = time.time()
print(reduce(lambda x, y: x + y, map(count_songs, df_chunks,)))
print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.')

2650
Время выполнения: 174.13 секунд.


In [ ]:
from joblib import Parallel, delayed
from functools import reduce
import csv


year_col = 3
views_col = 4


def count_songs(row):
    year = row[year_col].strip()
    views = row[views_col].strip()
    return 1 if year == '2022' and int(views) > 5_000 else 0


n = 0
start_time = time.time()
with open(dataset_path, "r", encoding='utf8') as file:
    print(reduce(lambda x, y: x + y, Parallel(n_jobs=2)(delayed(count_songs)(row) for row in csv.reader(file))))
print(f'Время выполнения: {round(time.time() - start_time, 2)} секунд.')

2650
Время выполнения: 354.54 секунд.


In [ ]:
import multiprocessing
import time

def square_sum(start, end):
    return sum(i * i for i in range(start, end))

def parallel_square_sum(n):
    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=num_processes)
    chunk_size = n // num_processes
    ranges = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_processes)]

    result = sum(pool.starmap(square_sum, ranges))
    pool.close()
    pool.join()
    return result

if __name__ == "__main__":
    n = 10**8
    start_time = time.time()
    result = parallel_square_sum(n)
    print(f"Result: {result}")
    print(f"Execution Time: {time.time() - start_time}")


Result: 333333328333333350000000
Execution Time: 16.455875396728516


In [ ]:
from joblib import Parallel, delayed
import time

def square_sum(start, end):
    return sum(i * i for i in range(start, end))

def parallel_square_sum(n, num_jobs):
    chunk_size = n // num_jobs
    ranges = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_jobs)]

    result = sum(Parallel(n_jobs=num_jobs)(delayed(square_sum)(start, end) for start, end in ranges))
    return result

if __name__ == "__main__":
    n = 10**8
    num_jobs = 2  # Задаем количество задач
    start_time = time.time()
    result = parallel_square_sum(n, num_jobs)
    print(f"Result: {result}")
    print(f"Execution Time: {time.time() - start_time}")


Result: 333333328333333350000000
Execution Time: 15.372305631637573


In [ ]:
import time

def square_sum(n):
    return sum(i * i for i in range(1, n+1))

if __name__ == "__main__":
    n = 10**8
    start_time = time.time()
    result = square_sum(n)
    print(f"Result: {result}")
    print(f"Execution Time: {time.time() - start_time}")


Result: 333333338333333350000000
Execution Time: 13.919626951217651


In [ ]:
import time
import random

def power(numbers, exponent):
    return [x**exponent for x in numbers]

if __name__ == "__main__":
    numbers = [random.randint(1, 100) for _ in range(10000000)]  # Большой список чисел
    exponent = 10  # Степень

    start_time = time.time()
    results = power(numbers, exponent)
    end_time = time.time()

    print(f"Sequential Execution Time: {end_time - start_time} seconds")


Sequential Execution Time: 4.23363471031189 seconds


In [ ]:
import multiprocessing
import time
import random

def power(number, exponent):
    return number**exponent

def parallel_power(numbers, exponent):
    pool = multiprocessing.Pool()
    results = pool.starmap(power, [(number, exponent) for number in numbers])
    pool.close()
    pool.join()
    return results

if __name__ == "__main__":
    numbers = [random.randint(1, 100) for _ in range(1000000)]  # Большой список чисел
    exponent = 10  # Степень

    start_time = time.time()
    results = parallel_power(numbers, exponent)
    end_time = time.time()

    print(f"Parallel Execution Time: {end_time - start_time} seconds")


Parallel Execution Time: 2.6723098754882812 seconds
